### Given a topic research the latest news

In [1]:
from dotenv import load_dotenv

load_dotenv()

False

In [14]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "dreamlike-art/dreamlike-photoreal-2.0", torch_dtype=torch.float16
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
pipe = pipe.to("cuda")

In [21]:
with open("scriptwriter_out.txt", "r") as file:
    # Read all lines from the file
    lines = file.readlines()

# Extract lines starting with "[Scene"
scene_lines = [line.strip() for line in lines if line.startswith("[Scene")]

# Print the extracted lines
for line in scene_lines:
    print(line)

[Scene 1] - "Aerial shot of the federal prison medical center in Butner, North Carolina. The sun is setting, casting long shadows across the prison complex. The mood is somber."
[Scene 2] - "Cut to archival footage of Kaczynski’s Montana shack. The camera zooms in on the remnants of the solitary building, surrounded by wilderness."
[Scene 3] - "Transition to scenes of Americans mailing packages and boarding airplanes, hinting at the substantial changes in these processes."
[Scene 4] - "Switch to images of the Washington Post and the New York Times front pages with the headline of Kaczynski’s manifesto."
[Scene 5] - "End with a montage of images from Kaczynski’s arrest: his Montana cabin, journals, coded diary, and two completed bombs. It ends with his mug shot, the image of a bearded hermit, a symbol of his seclusion."


In [32]:
images = []
for line in scene_lines:
    prompt = line
    image = pipe(prompt).images[0]
    images.append(image)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [38]:
for k, i in enumerate(images):
    n = k + 1
    name = "scene_{}.png".format(n)
    i.save(name)
    print(name)

scene_1.png
scene_2.png
scene_3.png
scene_4.png
scene_5.png
